In [1]:
import pandas as pd
import numpy as np

In [2]:
# CSV dosyasını oku
df = pd.read_csv("data/VT_Data.csv")

In [3]:
# 18 yaşından küçük kisileri çıkar
df = df[df['age'] >= 18]

In [4]:
# Veride herhangi bir eksik veri var mı?
print(df.isnull().any().any())

True


In [5]:
# Sütun bazında eksik veri sayısını göster
print(df.isnull().sum())

gender                  0
height                  0
weight                  0
age                     0
shoulder_c             18
chest_c                18
right_arm_c            18
left_arm_c             18
waist_c                 0
hip_c                   0
upper_leg_c            18
lower_leg              18
resting_pulse           0
chronic_conditions    143
surgeries             141
sports_experience       0
body_fat                0
goals                   0
program                 0
difficulty              0
dtype: int64


In [6]:
# chronic_conditions sütunundaki boş hücreleri 0 ile doldur
df['chronic_conditions'] = df['chronic_conditions'].fillna(0)
# surgeries sütunundaki boş hücreleri 0 ile doldur
df['surgeries'] = df['surgeries'].fillna(0)

# Veride baska bir eksik veri kaldi mı?
print(df.isnull().any().any())

True


In [7]:
# 'gender' sütunundaki değerleri 0 ve 1 olarak kodla
df['gender'] = df['gender'].map({'K': 0, 'E': 1})

In [8]:
# sports_experience sütununu güncellemek için
experience_mapping = {
    "No experience": 0,
    "Some experience": 1,
    "Athlete": 2
}
df['sports_experience'] = df['sports_experience'].map(experience_mapping)

In [9]:
# goals sütununu güncellemek için
goals_mapping = {
    "Improving overall health": "0",
    "Enhancing muscle growth": "1",
    "Improving physical appearance": "2",
    "Reducing body weight": "3"
}
df['goals'] = df['goals'].map(goals_mapping)

In [10]:
# BMI hesaplama fonksiyonu
def calculate_bmi(weight, height):
    if pd.notnull(weight) and pd.notnull(height) and height > 0:
        return round(weight / (height / 100) ** 2, 2)  # BMI hesaplama ve 2 basamağa yuvarlama
    return np.nan  # Eksik veriler için NaN döner

# BMI sütununu oluştur
df['BMI'] = df.apply(lambda row: calculate_bmi(row['weight'], row['height']), axis=1)

In [11]:
# WHR hesaplama fonksiyonu
def calculate_waist_to_hip_ratio(waist, hip):
    if pd.notnull(waist) and pd.notnull(hip) and hip > 0:
        return round(waist / hip, 2)  # WHR hesaplama ve 2 basamağa yuvarlama
    return np.nan  # Eksik veriler için NaN döner

# WHR sütununu oluştur
df['W/Hip_Ratio'] = df.apply(lambda row: calculate_waist_to_hip_ratio(row['waist_c'], row['hip_c']), axis=1)

In [12]:
# Bel/Boy oranı hesaplama fonksiyonu
def calculate_waist_to_height_ratio(waist, height):
    if pd.notnull(waist) and pd.notnull(height) and height > 0:
        return round(waist / height, 2)  # Bel/Boy oranı hesaplama ve 2 basamağa yuvarlama
    return np.nan  # Eksik veriler için NaN döner

# Bel/Boy oranı (Waist-to-Height Ratio) sütununu oluştur
df['W/Height_Ratio'] = df.apply(lambda row: calculate_waist_to_height_ratio(row['waist_c'], row['height']), axis=1)


In [13]:
# 'program' ve 'difficulty' kombinasyonlarını listele
unique_combinations = df[['program', 'difficulty']].drop_duplicates()
print(unique_combinations)

    program  difficulty
0      Faz1         1.0
1   Circuit         1.0
3      Faz2         1.0
4   Circuit         2.0
6      Faz1         2.0
12     Faz2         2.0
15     Faz1         3.0
19     Faz2         3.0
20     Faz3         1.0
22     Faz3         3.0
36  Circuit         3.0
48     Faz3         2.0


In [14]:
# 'program' ve 'difficulty' değerlerini birleştirip, 12 farklı opsiyona ayıran fonksiyon
def create_combined_column(row):
    combined_mapping = {
        ("Circuit", 1.0): "0",
        ("Circuit", 2.0): "1",
        ("Circuit", 3.0): "2",
        ("Faz1", 1.0): "3",
        ("Faz1", 2.0): "4",
        ("Faz1", 3.0): "5",
        ("Faz2", 1.0): "6",
        ("Faz2", 2.0): "7",
        ("Faz2", 3.0): "8",
        ("Faz3", 1.0): "9",
        ("Faz3", 2.0): "10",
        ("Faz3", 3.0): "11",
    }
    return combined_mapping.get((row['program'], row['difficulty']), None) 

# Yeni sütun oluşturma
df['Training_Program'] = df.apply(create_combined_column, axis=1)

# Her bir benzersiz seçeneğin sayısını hesapla
option_counts = df['Training_Program'].value_counts()

print(option_counts)

Training_Program
0     27
4     22
3     13
6     13
7     12
8     12
1     11
5      9
9      8
11     7
2      7
10     7
Name: count, dtype: int64


In [15]:
# Outlier (aykırı değer) tespiti için bir fonksiyon tanımlıyoruz
def detect_outliers(df, column):
    Q1 = df[column].quantile(0.25)  # 1. çeyrek
    Q3 = df[column].quantile(0.75)  # 3. çeyrek
    IQR = Q3 - Q1  # Çeyrekler arası fark

    lower_bound = Q1 - 1.5 * IQR  # Alt sınır
    upper_bound = Q3 + 1.5 * IQR  # Üst sınır

    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]  # Aykırı değerler
    return outliers

# Sayısal sütunları tespit et
numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns

# Her sütun için outlier'ları kontrol eden döngü
for column in numeric_columns:
    outliers = detect_outliers(df, column)
    print(f"Sütun: {column}")
    print(f"Aykırı Değer Sayısı: {len(outliers)}")
    if not outliers.empty:
        print(outliers)
    else:
        print("Aykırı değer bulunamadı.")
    print("-" * 50)

Sütun: gender
Aykırı Değer Sayısı: 0
Aykırı değer bulunamadı.
--------------------------------------------------
Sütun: height
Aykırı Değer Sayısı: 1
    gender  height  weight  age  shoulder_c  chest_c  right_arm_c  left_arm_c  \
22       1   206.0    95.0   33       117.5     95.0         34.5        36.0   

    waist_c  hip_c  ...  surgeries  sports_experience  body_fat  goals  \
22     88.0  105.0  ...        0.0                  2      18.0      2   

    program  difficulty    BMI W/Hip_Ratio W/Height_Ratio  Training_Program  
22     Faz3         3.0  22.39        0.84           0.43                11  

[1 rows x 24 columns]
--------------------------------------------------
Sütun: weight
Aykırı Değer Sayısı: 2
     gender  height  weight  age  shoulder_c  chest_c  right_arm_c  \
17        1   184.0   137.0   32       142.0    126.0         45.0   
128       1   185.0   130.0   35         NaN      NaN          NaN   

     left_arm_c  waist_c  hip_c  ...  surgeries  sports_expe

In [16]:
# Duplicate değerleri kontrol et
duplicates = df[df.duplicated()]
print(f"Duplicate Satır Sayısı: {len(duplicates)}")

Duplicate Satır Sayısı: 0


In [17]:
print(df.columns)

Index(['gender', 'height', 'weight', 'age', 'shoulder_c', 'chest_c',
       'right_arm_c', 'left_arm_c', 'waist_c', 'hip_c', 'upper_leg_c',
       'lower_leg', 'resting_pulse', 'chronic_conditions', 'surgeries',
       'sports_experience', 'body_fat', 'goals', 'program', 'difficulty',
       'BMI', 'W/Hip_Ratio', 'W/Height_Ratio', 'Training_Program'],
      dtype='object')


In [18]:
# Kullanılacak sütunların listesi
required_columns = ['gender', 'height', 'weight', 'age', 'resting_pulse', 'chronic_conditions', 'surgeries',
       'sports_experience', 'body_fat', 'goals', 'BMI', 'W/Hip_Ratio', 'W/Height_Ratio', 'Training_Program']

df = df[required_columns]

In [19]:
# Güncellenmiş veri çerçevesini CSV'ye yaz
df.to_csv("data/processed_VT_data.csv",  index=False)